<a href="https://colab.research.google.com/github/NH0917/amex/blob/main/model3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#!pip install catboost

In [ ]:
import polars as pl
import seaborn as sns
import matplotlib.pyplot as plt
import unicodedata
import numpy as np
import pandas as pd
import lightgbm as lgb
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
from sklearn.model_selection import StratifiedKFold
#from catboost import Pool,CatBoostClassifier
from sklearn.metrics import roc_auc_score
from imblearn.under_sampling import RandomUnderSampler
import itertools
from sklearn.preprocessing import LabelEncoder
import warnings
warnings.filterwarnings("ignore")

In [ ]:
train_path = "/content/drive/MyDrive/signate/cup/data/train.csv"
test_path = "/content/drive/MyDrive/signate/cup/data/test.csv"

In [ ]:
train = pl.read_csv(train_path)
test = pl.read_csv(test_path)

In [ ]:
age_map ={
    "一":10,
    "二":20,
    "三":30,
    "四":40,
    "五":50,
    "六":60,
    "七":70,
    "八":80,
    "九":90,
    "十":10,
    "1":10,
    "2":20,
    "3":30,
    "4":40,
    "5":50,
    "6":60,
    "7":70,
    "8":80,
    "9":90,
    "１":10,
    "２":20,
    "３":30,
    "４":40,
    "５":50,
    "６":60,
    "７":70,
    "８":80,
    "９":90,
}

gender_map ={
    "female":0,
    "male":1
}

customer_info1_map = {
    "独身":1,
"離婚済み":1,
"未婚":1,
"結婚済み":0}

customer_info2_map = {
    "車所持":1,
"車なし":0,
"自家用車なし":0,
"車未所持":0,
"自動車未所有":0,
"車あり":1,
"自家用車あり":1,
"乗用車なし":0,
"自動車所有":1,
"車保有":1,
"車保有なし":0,
"乗用車所持":1
}

customer_info3_map ={
    "子供の数不明":0,
"子の数不詳":0,
"2児":1,
"非育児家庭":0,
"子供有り(2人)":1,
"子供無し":0,
"こども2人":1,
"不明":0,
"子供なし":0,
"こども1人":1,
"こども3人":1,
"1児":1,
"わからない":0,
"子供3人":1,
"子供有り":1,
"子供有り(1人)":1,
"3児":1,
"子供1人":1,
"子育て状況不明":0,
"子供2人":1,
"無子":0,
"子供有り(3人)":1,
"子供ゼロ":0

}

product_map ={
    "basiс":"basic",
    "вasic":"basic",
    "superdeluxe":"superdeluxe",
    "delu×e":"deluxe",
    "βasic":"basic",
    "de|uxe":"deluxe",
    "kıոg":"king",
    "տuperdeluxe":"superdeluxe",
    "standard":"standard",
    "kiոg":"king",
    "ꭰeluxe":"deluxe",
    "basιc":"basic",
    "superdεluxe":"superdeluxe",
    "superꭰeluxe":"superdeluxe",
    "𐊡asic":"basic",
    "king":"king",
    "ꓢuperdeluxe":"superdeluxe",
    "deluxε":"deluxe",
    "baꓢic":"basic",
    "baտic":"basic",
    "ѕuperdeluxe":"superdeluxe",
    "basıς":"basic",
    "տtandard":"standard",
    "basiς":"basic",
    "basic":"basic",
    "staոdard":"standard",
    "deluxe":"deluxe",
    "stanᗞard":"standard",
    "ᗞeluxe":"deluxe",
    "βasıc":"basic",
    "bαsic":"basic",
    "superde|uxe":"superdeluxe",
    "standarꭰ":"standard",
    "de|u×e":"deluxe",
    "kıng":"king",
    "ꓢtandard":"standard",
    "ꓢuperde|uxe":"superdeluxe",
    "superᗞeluxe":"superdeluxe",
    "ѕtandard":"standard",
    "βasiс":"basic",
    "dεluxε":"deluxe",
    "basıc":"basic",
    'stαndard':"standard",
    'superdelu×e':"superdeluxe",
    'supεrꭰeluxe':"superdeluxe",
    'ꭰe|uxe':"deluxe",
    'basιс':"basic",
    'baѕic':"basic"
}

typeofcontact_map = {
    "Self Enquiry":0,
    "Company Invited":1,
}

In [ ]:
def preprocessing(df):
  #Age cleansing
  df = df.with_columns(pl.col("Age").fill_null(0))
  df = df.with_columns(pl.col("Age").str.slice(0,1))
  df = df.with_columns(pl.col("Age").replace(age_map))
  df = df.with_columns(pl.col("Age").cast(pl.Int64))

  #cvt duration of pitch sec->min
  df = df.with_columns(
    pl.when(
    pl.col("DurationOfPitch").str.contains("秒"))
    .then(pl.lit("Y").alias("DurationFlag"))
    .otherwise(pl.lit("N").alias("DurationFlag"))
)

  df = df.with_columns(pl.col("DurationOfPitch").str.replace("秒",""))
  df = df.with_columns(pl.col("DurationOfPitch").str.replace("分",""))
  df = df.with_columns(pl.col("DurationOfPitch").cast(pl.Int64))
  df = df.with_columns(
      pl.when(pl.col("DurationFlag")=="Y").then(pl.col("DurationOfPitch")/60).otherwise(pl.col("DurationOfPitch"))
  )

  df = df.with_columns(pl.col("DurationOfPitch")).fill_null(0)

  #cvt gender to one-hot
  df = df.with_columns(pl.col("Gender").map_elements(lambda x: unicodedata.normalize("NFKC",x)).str.to_lowercase().str.replace(" ",""))
  df = df.with_columns(pl.col("Gender").replace(gender_map))

  #cvt customer_info
  df = df.with_columns(pl.col("customer_info").str.replace_all(",|/|／|、|　|\t|\n| ","-"))
  df = df.with_columns(pl.col("customer_info").str.split("-"))
  df = df.with_columns((pl.col("customer_info").list.get(0)).alias("customer_info1"))
  df = df.with_columns((pl.col("customer_info").list.get(1)).alias("customer_info2"))
  df = df.with_columns((pl.col("customer_info").list.get(2)).alias("customer_info3"))

  df = df.with_columns(pl.col("customer_info1").replace(customer_info1_map))
  df = df.with_columns(pl.col("customer_info2").replace(customer_info2_map))
  df = df.with_columns(pl.col("customer_info3").replace(customer_info3_map))

  #NumberOfFollowups
  df = df.with_columns(
    pl.when(pl.col("NumberOfFollowups")>=100).then(pl.col("NumberOfFollowups")/100).otherwise(pl.col("NumberOfFollowups")
  )
  )

  #cvt number of trips
  df = df.with_columns(
    pl.when(
        pl.col("NumberOfTrips").str.contains("半年に")
    ).then(pl.lit(2).alias("MultiNum")
    ).when(pl.col("NumberOfTrips").str.contains("四半期に")
    ).then(pl.lit(4).alias("MultiNum")
    ).otherwise(pl.lit(1).alias("MultiNum")
        )
    )

  #product
  df = df.with_columns(pl.col("ProductPitched").map_elements(lambda x: unicodedata.normalize("NFKC",x)).str.to_lowercase().str.replace(" ",""))
  df = df.with_columns(
      pl.col("ProductPitched").replace(product_map).fill_null("other")
  )

  df = df.with_columns(pl.col("NumberOfTrips").str.extract(r".*(\d{1,2})"))
  df = df.with_columns(pl.col("NumberOfTrips").cast(pl.Int64()))
  df = df.with_columns((pl.col("NumberOfTrips")*pl.col("MultiNum")).alias("NumberOfTrips_cvt"))

  df = df.with_columns(
    pl.when(pl.col("MonthlyIncome").str.contains("万")
    ).then(pl.lit("Yes").alias("MonthlyIncomeFlag")).otherwise(pl.lit("No").alias("MonthlyIncomeFlag")))
  df = df.with_columns(
      pl.when(pl.col("MonthlyIncome").str.contains("万")
  ).then(pl.col("MonthlyIncome").str.extract(r"月収(\d{1,3})")).otherwise(pl.col("MonthlyIncome"))
  )
  df = df.with_columns(
      pl.when(pl.col("MonthlyIncomeFlag").eq("Yes")).then(pl.col("MonthlyIncome").cast(pl.Float64())*10000).otherwise(pl.col("MonthlyIncome").cast(pl.Float64())
  ))
  df = df.drop("MonthlyIncomeFlag")

  df = df.with_columns(pl.col("TypeofContact").replace(typeofcontact_map).fill_null(2))

  df = df.fill_null(0)

  return df

In [ ]:
feature_col = ["Age","CityTier","Gender","NumberOfPersonVisiting","NumberOfFollowups","PreferredPropertyStar","ProductPitched","NumberOfTrips_cvt","Passport","PitchSatisfactionScore","customer_info1","customer_info2","customer_info3","MonthlyIncome","DurationOfPitch"]

In [ ]:
train = preprocessing(train)
train = train.select(feature_col+["ProdTaken"])
train = train.to_pandas()

test = preprocessing(test)
test = test.select(feature_col)
test = test.to_pandas()

In [ ]:
def create_monthlyincome_feature(df):
  df["MonthlyIncome_ceil_10_000"] = df["MonthlyIncome"].apply(lambda x: (x//10_000)*10_000).astype(int)
  df["MonthlyIncome_ceil_100_000"] = df["MonthlyIncome"].apply(lambda x: (x//100_000)*100_000).astype(int)
  return df

In [ ]:
train = create_monthlyincome_feature(train)
test = create_monthlyincome_feature(test)
feature_col.extend(["MonthlyIncome_ceil_10_000","MonthlyIncome_ceil_100_000"])

In [ ]:
def target_encoding(train_df,val_df,col):
  label_mean = train_df.groupby(col)["ProdTaken"].mean().to_dict()
  train_df[f"{col}_target_enc"] = train_df[col].map(label_mean)
  val_df[f"{col}_target_enc"] = val_df[col].map(label_mean)
  return train_df,val_df

In [ ]:
def create_customer_feature(df):
  df["customer_info_1_2_3"] = df["customer_info1"].astype(str)+df["customer_info2"].astype(str)+df["customer_info3"].astype(str)
  df["Age_Gender"] = df["Age"].astype(str)+df["Gender"].astype(str)
  create_col = ["customer_info_1_2_3","Age_Gender"]
  return df,create_col

In [ ]:
train,create_col = create_customer_feature(train)
test,_ = create_customer_feature(test)

target_col = ["ProductPitched","Age","Gender","customer_info1","customer_info2","customer_info3"]
target_col += create_col

In [ ]:
params ={
    "objective":"binary",
    "metric":"binary_logloss",
    "random_state":0,
    "verbosity":-1,
    "boosting_type":"dart",
}

In [ ]:
random_state = 0

auc_list = []
models_list = []

oof_dfs = pd.DataFrame()


skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=random_state)
for i,(train_index,val_index) in enumerate(skf.split(train,train["ProdTaken"])):

    train_df = train.loc[train_index,:].reset_index(drop=True)
    val_df = train.loc[val_index,:].reset_index(drop=True)

    pos_count = int(train_df["ProdTaken"].value_counts()[1]*0.7)
    neg_count = int(train_df["ProdTaken"].value_counts()[0]*0.7)

    for col in target_col:
      train_df,val_df = target_encoding(train_df,val_df,col)

    train_df.drop(columns=target_col,axis=1,inplace=True)
    val_df.drop(columns=target_col,axis=1,inplace=True)

    oof_df = pd.DataFrame()
    for seed in range(5):
      rus = RandomUnderSampler(random_state=seed,sampling_strategy={0:pos_count,1:pos_count})
      train_X,train_y = rus.fit_resample(train_df.drop(columns="ProdTaken",axis=1),train_df["ProdTaken"])
      train_dataset = lgb.Dataset(train_X,train_y)
      val_dataset = lgb.Dataset(val_df.drop(columns="ProdTaken",axis=1),val_df["ProdTaken"])
      #model = lgb.train(params,train_dataset,valid_sets=[val_dataset],num_boost_round=70,callbacks=[lgb.early_stopping(stopping_rounds=10,verbose=True)])
      model = lgb.train(params,train_dataset,valid_sets=[val_dataset],num_boost_round=40)
      y_pred = model.predict(val_df.drop(columns="ProdTaken",axis=1),num_iteration=model.best_iteration)
      oof_df[f"seed_{seed}"] = y_pred
      model.save_model(f"model_{i}_{seed}",num_iteration=model.best_iteration)
    val_df["predict"] = oof_df.mean(axis=1)
    oof_dfs = pd.concat([oof_dfs,val_df[["predict","ProdTaken"]]],axis="index")


In [ ]:
roc_auc_score(oof_dfs["ProdTaken"],oof_dfs["predict"])

0.8240095654138735

In [ ]:
roc_auc_score(oof_dfs["ProdTaken"],oof_dfs["predict"])

0.8214588332131828

In [ ]:
oof_dfs

,predict,ProdTaken
0,0.946366,1
1,0.360661,0
2,0.466282,0
3,0.758636,1
4,0.581096,0
...,...,...
692,0.019934,0
693,0.041756,0
694,0.073267,0
695,0.593588,0


In [ ]:
neg_count

2394

In [ ]:
features = []
for i in range(5):
    model = lgb.Booster(model_file = f"model_{i}_{i}")
    feature = model.feature_importance(importance_type="gain")
    features.append(feature)

In [ ]:
temp = pd.DataFrame(features,columns=usecol)
temp = temp.mean().reset_index(drop=False).rename(columns={
    "index":"feature",
    0:"importance"
})

ValueError: 10 columns passed, passed data had 19 columns

In [ ]:
sns.barplot(x="importance",y="feature",data=temp.sort_values("importance",ascending=False).head(20))

In [ ]:
sub = pd.read_csv(test_path)

In [ ]:
usecol = set(train_df.drop(columns="ProdTaken",axis=1).columns)-set(test.columns)

In [ ]:
usecol

{'Age_Gender_target_enc',
 'Age_target_enc',
 'Gender_target_enc',
 'ProductPitched_target_enc',
 'customer_info1_target_enc',
 'customer_info2_target_enc',
 'customer_info3_target_enc',
 'customer_info_1_2_3_target_enc'}

In [ ]:
usecol=['Age_Gender',
 'Age',
 'Gender',
 'ProductPitched',
 'customer_info1',
 'customer_info2',
 'customer_info3',
 'customer_info_1_2_3']

In [ ]:
train["Age_Gender"] = train["Age"].astype(str)+train["Gender"].astype(str)
train["customer_info_1_2_3"] = train["customer_info1"].astype(str)+train["customer_info2"].astype(str)+train["customer_info3"].astype(str)

In [ ]:
train

,Age,CityTier,Gender,NumberOfPersonVisiting,NumberOfFollowups,PreferredPropertyStar,ProductPitched,NumberOfTrips_cvt,Passport,PitchSatisfactionScore,customer_info1,customer_info2,customer_info3,MonthlyIncome,DurationOfPitch,ProdTaken,MonthlyIncome_ceil_10_000,MonthlyIncome_ceil_100_000,customer_info_1_2_3,Age_Gender
0,50,2,1,1.0,4.0,3.0,basic,5,1,4,1,0,0,253905.0,15.0,1,250000,200000,100,501
1,50,1,1,1.0,4.0,3.0,standard,2,1,4,1,1,0,404475.0,14.0,0,400000,400000,110,501
2,0,1,0,1.0,3.0,3.0,basic,4,0,4,0,0,0,278145.0,10.0,1,270000,200000,000,00
3,30,2,0,1.0,3.0,4.0,standard,1,0,5,1,1,0,326805.0,18.0,0,320000,300000,110,300
4,40,3,0,1.0,3.0,4.0,basic,4,0,4,1,1,0,258435.0,17.0,1,250000,200000,110,400
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3484,40,2,1,2.0,3.0,3.0,basic,3,0,1,1,1,0,258900.0,26.0,1,250000,200000,110,401
3485,40,1,1,3.0,3.0,5.0,basic,5,0,3,0,1,1,260415.0,9.0,0,260000,200000,011,401
3486,30,1,0,3.0,2.0,3.0,standard,5,0,4,1,1,1,317340.0,14.0,0,310000,300000,111,300
3487,50,2,1,3.0,6.0,3.0,king,7,1,4,0,0,1,527910.0,15.0,1,520000,500000,001,501


In [ ]:
for col in usecol:
  target_enc = train.groupby(col)["ProdTaken"].mean().to_dict()
  test[f"{col}_target_enc"] = test[col].map(target_enc)

In [ ]:


pred_df = pd.DataFrame()
pred_df["id"] = sub["id"].copy()

usecol = ['CityTier', 'NumberOfPersonVisiting', 'NumberOfFollowups',
       'PreferredPropertyStar', 'NumberOfTrips_cvt', 'Passport',
       'PitchSatisfactionScore', 'MonthlyIncome', 'DurationOfPitch',
       'MonthlyIncome_ceil_10_000', 'MonthlyIncome_ceil_100_000',
       'ProductPitched_target_enc', 'Age_target_enc', 'Gender_target_enc',
       'customer_info1_target_enc', 'customer_info2_target_enc',
       'customer_info3_target_enc', 'customer_info_1_2_3_target_enc',
       'Age_Gender_target_enc']
for i in range(5):
  for j in range(5):
    model = lgb.Booster(model_file = f"model_{i}_{j}")
    y_pred = model.predict(test[usecol])
    pred_df[f"model_{i}_{j}"] = y_pred

In [ ]:
for col in pred_df.columns[1:]:
  pred_df[col] = pred_df[col].rank(ascending=True)

In [ ]:
pred_df["pre"] = pred_df.iloc[:,1:].mean(axis=1)

In [ ]:
pred_df[["id","pre"]].to_csv("submission.csv",index=False,header=False)

In [ ]:
pred_df["pred"] = pred_df.iloc[:,1:].mean(axis=1)

In [ ]:
pred_df[["id","pred"]].to_csv("submission.csv",index=False,header=False)

In [ ]:
pred_df

In [ ]:
pred_df